In [1]:
# Dependencies
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import requests
import json
import alpha_vantage
from config import apiKey
from alpha_vantage.timeseries import TimeSeries

def one_year_data(stock_ticker): 
    yesterday = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
    past_year = datetime.strftime(datetime.now() - timedelta(366), '%Y-%m-%d')
    

    url = "https://www.alphavantage.co/query?"
    query_url = f"{url}function=TIME_SERIES_DAILY&symbol={stock_ticker}&outputsize=full&apikey={apiKey}" 

    response = requests.get(query_url).json() 

    df = pd.DataFrame(response["Time Series (Daily)"]) 

    df_transposed = df.T # or df1.transpose()

    df_filtered = df_transposed[yesterday : past_year] 

    df_filtered.rename(columns={'4. close':'close', '3. low':'low', '2. high':'high', '5. volume':'volume'}, inplace=True)
    
    df_filtered.drop('1. open', axis=1, inplace=True)
    
    df_filtered = df_filtered[["close", "low", "high", "volume"]]
    
    
    return df_filtered  

one_year_data("AAPL")    

ModuleNotFoundError: No module named 'config'

In [ ]:
# Dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM